# How to add message history

LangGraph allows you to manage message history (and other stateful data) in multi-turn chatbot applications by:

* Storing conversation state (e.g., user messages, responses).
* Supporting memory across multiple interactions or threads (e.g., conversations with different users).
* Using persistence to store the state in memory, SQLite, Postgres, or Redis.

# How It Works
  1 . Initialize the Chat Model: Use your preferred Large Language Model (e.g., OpenAI, Anthropic). This model will handle chat interactions.

In [5]:
import getpass
import os
from langchain_openai import ChatOpenAI

os.environ["OPENAI_API_KEY"] = getpass.getpass()
llm = ChatOpenAI(model="gpt-4o-mini")


2. Define a Graph to Handle State: A StateGraph manages the message history and interaction flow. The graph has:

* A state schema: Defines the structure of the stored data (e.g., messages).
* Nodes: Represent steps in the workflow (e.g., calling the model).
* Edges: Connect the steps.

In [7]:
pip install langgraph

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

# Define a graph with a schema for storing messages
workflow = StateGraph(state_schema=MessagesState)

# Define a function to call the model and update the message history
def call_model(state: MessagesState):
    response = llm.invoke(state["messages"])  # Get response from LLM
    return {"messages": response}  # Add response to message history

# Add the function as a node in the graph
workflow.add_edge(START, "model")  # Connect START to the model node
workflow.add_node("model", call_model)

# Add memory to store the state between runs
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)  # Compile the graph with memory


3. Run the Application: Invoke the graph by:

* Passing messages to the model.
* Using a thread_id to manage separate conversations

In [9]:
from langchain_core.messages import HumanMessage

# First query: Introduce yourself
config = {"configurable": {"thread_id": "abc123"}}  # Conversation ID
query = "Hi! I'm Bob."
input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)

# Print the AI response
output["messages"][-1].pretty_print()


================================== Ai Message ==================================

Hi Bob! How can I assist you today?


4. Handle Multiple Turns: Pass new queries while keeping the same thread_id to retain memory:



In [10]:
query = "What's my name?"
input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)

# Print the AI response
output["messages"][-1].pretty_print()


================================== Ai Message ==================================

Your name is Bob. How can I help you today, Bob?


5. Separate Threads: Use a new thread_id to start a fresh conversation with no memory:



In [11]:
config = {"configurable": {"thread_id": "abc234"}}
query = "What's my name?"
input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)

output["messages"][-1].pretty_print()


================================== Ai Message ==================================

I'm sorry, but I don't know your name. If you'd like to share it, feel free to do so!


6. Access and Update State:

  * View State:

In [12]:
state = app.get_state(config).values
print(state["messages"])


[HumanMessage(content="What's my name?", additional_kwargs={}, response_metadata={}, id='8a7f7a4b-28d8-4b61-b56a-c5ebe448625d'), AIMessage(content="I'm sorry, but I don't know your name. If you'd like to share it, feel free to do so!", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 11, 'total_tokens': 34, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0705bf87c0', 'finish_reason': 'stop', 'logprobs': None}, id='run-7202d73c-743a-401f-ae30-de677160b0fa-0', usage_metadata={'input_tokens': 11, 'output_tokens': 23, 'total_tokens': 34, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]


* Update State: Add messages manually:


In [13]:
from langchain_core.messages import HumanMessage
app.update_state(config, {"messages": [HumanMessage("Test")]})


{'configurable': {'thread_id': 'abc234',
  'checkpoint_ns': '',
  'checkpoint_id': '1eface2d-feaa-6569-8002-bcb95b659f33'}}

# Why Use This Approach?
Persistence: Store and manage conversation history effortlessly.

Multi-user Support: Separate threads for different users.

Flexibility: Use in-memory storage or external databases.

# Simplified Workflow
Initialize the model.

Create a graph with StateGraph.

Add memory to the graph.

Run queries with a thread_id to manage conversation history.